In [49]:
import matplotlib.pyplot as pyp
import scipy.stats as st
import pandas as p
import numpy as n
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [50]:
def read_dataset(path, separ, en):
    return p.read_csv(path, sep = separ, encoding = en)

dataset = read_dataset("fullDataset.csv", ',', 'cp1252')
dataset.info()
dataset.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139483 entries, 0 to 139482
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  139483 non-null  int64  
 1   ID          139483 non-null  int64  
 2   Sex         139483 non-null  object 
 3   Age         139483 non-null  int64  
 4   Height      139483 non-null  int64  
 5   Weight      139483 non-null  int64  
 6   Team        139483 non-null  object 
 7   Year        139483 non-null  int64  
 8   Season      139483 non-null  object 
 9   Sport       139483 non-null  object 
 10  Medal       18968 non-null   object 
 11  Code        139483 non-null  object 
 12  GDP         139483 non-null  float64
 13  Population  139483 non-null  float64
dtypes: float64(2), int64(6), object(6)
memory usage: 14.9+ MB


,Unnamed: 0,ID,Sex,Age,Height,Weight,Team,Year,Season,Sport,Medal,Code,GDP,Population
0,0,1,M,24,180,80,China,1992,Summer,Basketball,NaN,CHN,4.269157e+11,1.164970e+09
1,1,2,M,23,170,60,China,2012,Summer,Judo,NaN,CHN,8.532230e+12,1.350695e+09
2,2,5,F,21,185,82,Netherlands,1988,Winter,Speed Skating,NaN,NLD,2.619105e+11,1.476009e+07
3,3,5,F,21,185,82,Netherlands,1988,Winter,Speed Skating,NaN,NLD,2.619105e+11,1.476009e+07
4,4,5,F,25,185,82,Netherlands,1992,Winter,Speed Skating,NaN,NLD,3.629629e+11,1.518417e+07


In [51]:
drop_list = ["Unnamed: 0", "Sex", "Age", "Height", "Weight", "Team", "Sport"]

for str in drop_list:
    dataset = dataset.drop(str, axis = 1)

dataset.info()
dataset.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139483 entries, 0 to 139482
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID          139483 non-null  int64  
 1   Year        139483 non-null  int64  
 2   Season      139483 non-null  object 
 3   Medal       18968 non-null   object 
 4   Code        139483 non-null  object 
 5   GDP         139483 non-null  float64
 6   Population  139483 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 7.4+ MB


,ID,Year,Season,Medal,Code,GDP,Population
0,1,1992,Summer,NaN,CHN,4.269157e+11,1.164970e+09
1,2,2012,Summer,NaN,CHN,8.532230e+12,1.350695e+09
2,5,1988,Winter,NaN,NLD,2.619105e+11,1.476009e+07
3,5,1988,Winter,NaN,NLD,2.619105e+11,1.476009e+07
4,5,1992,Winter,NaN,NLD,3.629629e+11,1.518417e+07
5,5,1992,Winter,NaN,NLD,3.629629e+11,1.518417e+07
6,5,1994,Winter,NaN,NLD,3.791303e+11,1.538284e+07
7,5,1994,Winter,NaN,NLD,3.791303e+11,1.538284e+07
8,6,1992,Winter,NaN,USA,6.520327e+12,2.565140e+08
9,6,1992,Winter,NaN,USA,6.520327e+12,2.565140e+08


In [52]:
dataset = dataset.groupby(["Year", "Season", "Code"]).agg(GDP = ("GDP", "mean"),
                                                          Population = ("Population", "mean"),
                                                          Members = ("ID","nunique"),
                                                          Medal = ("Medal", "count"))


dataset.info()

dataset.to_csv("check_r.csv", sep=',', encoding='utf-8')
dataset.head(10)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2258 entries, (1980, 'Summer', 'AFG') to (2016, 'Summer', 'ZWE')
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   GDP         2258 non-null   float64
 1   Population  2258 non-null   float64
 2   Members     2258 non-null   int64  
 3   Medal       2258 non-null   int64  
dtypes: float64(2), int64(2)
memory usage: 81.2+ KB


GDP   Population  Members  Medal
Year Season Code                                           
1980 Summer AFG   3.641723e+09   13356511.0       11      0
            AGO   5.934074e+09    8341289.0       11      0
            AND   4.464161e+08      36067.0        2      0
            AUS   1.500323e+11   14692000.0      120     13
            AUT   8.205891e+10    7549433.0       83      5
            BEL   1.268293e+11    9859242.0       59      1
            BEN   1.405252e+09    3717165.0        7      0
            BGR   1.983923e+10    8861535.0      264     89
            BRA   2.373935e+11  120694009.0      105      9
            BWA   1.060924e+09     897868.0        7      0

In [53]:
def kolm_smirn_check(dFrame, column):
    ks_statistic, p_value = st.kstest(dFrame[column], 'norm')
    if p_value > 0.05: return True
    return False

def pearson_check(dFrame, column):
    statistic, p_value = st.normaltest(dFrame[column])
    if p_value > 0.05: return True
    return False

print("#1 test - Kolmagorov-Smirnov test,\n#2 test - Pearson test\n")
for c in dataset.columns:
    print('{0:<15}'.format(c + ":"),f"#1 test - {kolm_smirn_check(dataset, c)}, #2 test - {pearson_check(dataset, c)};")

#1 test - Kolmagorov-Smirnov test,
#2 test - Pearson test

GDP:            #1 test - False, #2 test - False;
Population:     #1 test - False, #2 test - False;
Members:        #1 test - False, #2 test - False;
Medal:          #1 test - False, #2 test - False;


In [54]:
dataset.corr()

,GDP,Population,Members,Medal
GDP,1.000000,0.366452,0.547481,0.612213
Population,0.366452,1.000000,0.244177,0.243602
Members,0.547481,0.244177,1.000000,0.876835
Medal,0.612213,0.243602,0.876835,1.000000


In [55]:
first = dataset.iloc[:, :3]
second = dataset['Medal']
f_train, f_test, s_train, s_test = train_test_split(first, second, test_size=0.3)

In [56]:
par = ["GDP", "Population", "Members", ["GDP", "Population"],["GDP", "Members"],["Members", "Population"],["GDP", "Population","Members"]]

lin = []
pol = []
for i in range(len(par)):
    pol.append(make_pipeline(PolynomialFeatures(degree = 2), LinearRegression()))
    if i < 3: 
        lin.append(LinearRegression().fit(f_train[par[i]].to_numpy().reshape(-1,1), s_train))
        pol[i].fit(f_train[par[i]].to_numpy().reshape(-1,1), s_train)
    else:
        lin.append(LinearRegression().fit(f_train[par[i]], s_train))
        pol[i].fit(f_train[par[i]], s_train)

In [57]:
prediction = []

def predict_by(prediction, s):
    for i in range(len(par)):
        if i < 3:
            prediction.append(s[i].predict(f_test[par[i]].to_numpy().reshape(-1,1)))
        else:
            prediction.append(s[i].predict(f_test[par[i]]))
    return prediction

prediction = predict_by(prediction, lin)
prediction = predict_by(prediction, pol)

In [58]:
min_i = n.sum((n.array(prediction) - s_test.to_numpy())**2,axis = 1).argmin()
min= n.sum((n.array(prediction) - s_test.to_numpy())**2,axis = 1)
print(min)
print("Best model is:")
if (min_i < 7):
    print(f"linear model by {par[min_i]} params")
else:
    print(f"polynomial model by {par[min_i - 7]} params")

[221450.55421437 350741.96763879  83205.18288303 221169.86636785
  70443.17333861  82585.03056099  70462.92601931 211922.78666206
 356206.77551622  74615.8057501  218069.01356592 151229.52911311
  89691.49841281 156022.62182073]
Best model is:
linear model by ['GDP', 'Members'] params


In [59]:
# from mpl_toolkits.mplot3d import Axes3D

# params = ['GDP', 'Population']
# params_values = []

# for i in range(len(params)):
#     values = n.linspace(f_test[params[i]].min(), f_test[params[i]].max()).reshape(-1, 1)
#     params_values.append(values)


# fig = pyp.figure(figsize=(8, 8))
# ax = fig.gca(projection='3d')

# X_3d = params_values[0]
# Y_3d = params_values[1]

# XX, YY = n.meshgrid(X_3d, Y_3d)

# Z = []
# for i in range(len(Y_3d)):
#     temp = []
#     for j in range(len(X_3d)):
#         temp.append(pol[2].predict(n.array([X_3d[j], Y_3d[i]]).T)[0])
#     Z.append(temp)
    
# Z = n.array(Z)

# ax.set_title('Поліноміальна регресія Medals ~ GDP, Population')
# ax.set_xlabel('GDP')
# ax.set_ylabel('Population')
# ax.set_zlabel('Medals')
# ax.plot_surface(
#     XX, YY,
#     n.array(Z),
#     color='green',
#     alpha=0.5
# )
# ax.scatter(f_test['GDP'], f_test['Population'], s_test)
# pyp.show()

In [60]:
arr = pol[2].predict(f_test[par[2]].to_numpy().reshape(-1,1))
sum = 0
count = 0
for i in arr:
    print(i)
    count += 1
    sum += i

print(f"avg {sum / count}")

0.5369537649790315
0.5369537649790315
0.5369537649790315
3.2304540732117673
0.47622676755321314
1.2860694888019428
0.8590248321396935
0.7265091892691147
0.5369537649790315
0.9271262615781399
12.089304759657026
0.7265091892691147
0.5989098344069546
0.9271262615781399
0.6620949758369822
1.5953473666188693
3.6478902872139067
0.9964567630186911
0.5369537649790315
0.5369537649790315
0.5989098344069546
0.7921524747033517
11.733098012403394
3.6478902872139067
0.47622676755321314
4.54175817131921
0.5989098344069546
15.315557239652808
0.6620949758369822
0.5369537649790315
0.5369537649790315
1.1388049819061075
0.5369537649790315
39.883278267296774
6.702405225490226
2.0095988339373827
0.5369537649790315
1.1388049819061075
13.753074186402552
1.2118226993529728
1.4382502837061966
5.263780108486585
0.47622676755321314
0.7921524747033517
0.47622676755321314
5.514286663559214
2.0095988339373827
63.88016053695203
1.4382502837061966
0.5369537649790315
10.19100671386623
4.310916768280256
0.92712626157813